In [ ]:
import json
from vllm import LLM, SamplingParams

location_path = "./raw/meta-other.json"
review_path = "./raw/reviews-other.json"
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

In [ ]:
def load_location_dict(path):
    meta = {}
    drop_keys = {"latitude", "longitude", "url"}
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            obj = json.loads(line)
            for key in drop_keys:
                obj.pop(key, None)
            meta[obj["location_id"]] = obj
    return meta

def load_reviews_dict(path):
    meta = []
    drop_keys = {"user_id", "time"}
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            obj = json.loads(line)
            for key in drop_keys:
                obj.pop(key, None)
            meta.append(obj)
    return meta

def load_reviews_gen(path, start=0, end=1000):
    drop_keys = {"user_id", "time"}
    with open(path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f):
            if i < start:
                continue
            if end is not None and i >= end:
                break
            obj = json.loads(line)
            for key in drop_keys:
                obj.pop(key, None)
            yield obj

In [ ]:
llm = LLM(
    model=model_name,
    tensor_parallel_size=1,   
    dtype="fp16",              
    device="cuda"  
)